In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 
import os
from tqdm import tqdm
import json
import time
import random
import pickle

In [3]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

video = Input(shape=(None,64,48,3))
cnn_base = VGG16(input_shape=(64,48,3),
                 weights="imagenet",
                 include_top=False
                )
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = True
encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = LSTM(1024)(encoded_frames)
encoded_sequence = LSTM(512)(encoded_frames)
hidden_layer = Dense(512, activation="tanh")(encoded_sequence)
outputs = Dense(300, activation="tanh")(hidden_layer)
model = Model([video], outputs)

model.compile(loss="mean_squared_error",
              optimizer='adam',
              metrics=["mse"]) 

In [4]:
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, 64, 48, 3)] 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 512)         14714688  
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 300)               153900    
Total params: 17,230,444
Trainable params: 17,230,444
Non-trainable params: 0
_________________________________________________________________


In [5]:
ed=pd.read_csv('E:\capstone\code\WLASL-master\WLASL-master\start_kit\embedding_data.csv')

In [6]:
ed.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,gloss
0,-0.256340,0.374570,-0.004352,-0.031364,0.497100,0.500190,-0.316900,-0.029405,-0.223950,2.3810,...,-0.038286,-0.125790,0.34074,0.676660,0.034156,0.70225,-0.44008,0.159510,-0.401530,book
1,-0.063465,-0.160110,0.383710,-0.376820,-0.043688,0.167340,-0.358510,0.183800,-0.112990,2.0250,...,-0.317530,-0.162240,-0.36107,-0.247620,0.285030,-0.18506,-0.23737,0.916300,-0.054556,drink
2,-0.266570,-0.137170,0.235490,-0.267120,0.093324,0.175630,-0.333090,-0.817440,0.526550,1.5880,...,-0.167160,0.375520,0.57434,0.055414,-0.225450,0.30528,0.35608,-0.464960,0.139140,computer
3,0.417900,0.034876,-0.187950,-0.135520,0.135230,-0.173070,-0.101180,0.000775,0.099486,3.1386,...,0.179180,0.087482,-0.12456,-0.098174,-0.221110,-0.13060,0.26873,0.064549,-0.024694,before
4,-0.020407,-0.495840,0.024120,0.102360,0.199390,0.064525,-0.095029,-0.604110,-0.059009,2.2198,...,-0.605170,0.940230,-0.55998,-0.169490,0.068575,0.48710,0.37698,-0.420090,-0.031210,chair


In [7]:
df=pd.read_csv("E:\capstone\code\WLASL-master\WLASL-master\start_kit\data.csv")
df.head()

,gloss,video_id,xmin,ymin,xmax,ymax,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id
0,book,69241,385,37,885,720,25.0,-1,1,0,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/book.mp4,0
1,book,65225,190,25,489,370,25.0,-1,1,1,90,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0
2,book,68011,262,1,652,480,25.0,-1,1,2,110,valencia-asl,train,https://www.youtube.com/watch?v=0UsjUE-TXns,0
3,book,68208,123,19,516,358,25.0,60,1,3,113,lillybauer,train,https://www.youtube.com/watch?v=1QOYOZ3g-aY,0
4,book,68012,95,0,1180,720,25.0,-1,1,4,109,valencia-asl,train,https://www.youtube.com/watch?v=aGtIHKEdCds,0


In [8]:
words=df.gloss.unique()

In [9]:
len(words)

2000

In [17]:
x_train=[]
x_test=[]
x_val=[]
y_train=[]
y_test=[]
y_val=[]
y_words_train=[]
y_words_test=[]
y_words_val=[]


for word in tqdm(words[:50]): 

    for split in os.listdir("E:\capstone\code\WLASL-master\WLASL-master\start_kit\preprocessed_vid_by_word_48_64\{}".format(word)):
        try:
            for v in os.listdir("E:\capstone\code\WLASL-master\WLASL-master\start_kit\preprocessed_vid_by_word_48_64\{}\{}".format(word,split)):
                y_ind=ed[ed['gloss']==word].values[:,:-1].reshape((1,300))
                x_ind=np.load("E:\capstone\code\WLASL-master\WLASL-master\start_kit\preprocessed_vid_by_word_48_64\{}\{}\{}".format(word,split,v))
            
                if(split=="train"):
                    x_train.append(x_ind)
                    y_train.append(y_ind)
                    y_words_train.append(word)
                elif(split=="test"):
                    x_test.append(x_ind)
                    y_test.append(y_ind)
                    y_words_test.append(word)
                else:
                    x_val.append(x_ind)
                    y_val.append(y_ind)
                    y_words_val.append(word)
    
        except Exception as e:
            print("Error:" ,e ,"file",i['Key'])
        

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 38.28it/s]


In [18]:
ed_part=[]
for word in tqdm(words[:100]): 
    ed_part.append(ed[ed['gloss']==word].values.reshape((301)))

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1388.90it/s]


In [19]:
ed_part=np.asarray(ed_part)


In [20]:
len(x_train),len(y_train),len(x_test),len(y_test),len(x_val),len(y_val)

(680, 680, 132, 132, 158, 158)

In [21]:
x_train=np.array(x_train)
x_test=np.array(x_test)
x_val=np.array(x_val)
y_train=np.array(y_train)
y_test=np.array(y_test)
y_val=np.array(y_val)

In [22]:
np.asarray(y_train[0])

array([[-0.2563399970531464, 0.3745700120925903, -0.004352000076323748,
        -0.03136400133371353, 0.4970999956130981, 0.5001900196075441,
        -0.3169000148773193, -0.029404999688267708, -0.22394999861717224,
        2.38100004196167, -0.8131800293922424, 0.27068999409675604,
        0.3430800139904022, -0.07821600139141083, 0.26739999651908875,
        -0.5194100141525269, -0.31679001450538635, 0.8490999937057495,
        0.02203400060534477, -0.5310500264167786, 0.08698499947786331,
        -0.11784999817609787, 0.2147900015115738, 0.03904300183057785,
        -0.09996099770069122, 0.1139099970459938, -0.21740999817848208,
        -0.3270399868488312, -0.2949100136756897, -0.0860000029206276,
        -0.243819996714592, 0.09882400184869766, -0.022630000486969948,
        0.2913399934768677, 0.2726399898529053, 0.33285000920295715,
        0.045503001660108566, 0.09840700030326843, -0.185139998793602,
        -0.4682100117206574, 0.09139999747276306, -0.022836999967694286,
    

In [ ]:
error=[]
accuracy=[]
for epoch in range(10):
    for i in tqdm(range(x_train.shape[0])):
        x_train_ind=np.expand_dims(x_train[i],axis=0)
        met=model.fit(np.asarray(x_train_ind).astype(np.float32),np.asarray(y_train[i]).astype(np.float32),batch_size=1,epochs=1)
        error.append(met.history['mse'])
        if(i%5==0):
            plt.plot(error)
            plt.plot(accuracy)
        

  0%|                                                                                          | 0/680 [00:00<?, ?it/s]

1/1 [==============================] - 0s 2ms/step - loss: 0.0453 - mse: 0.0453


  0%|                                                                                  | 1/680 [00:03<34:46,  3.07s/it]

1/1 [==============================] - 0s 3ms/step - loss: 0.0386 - mse: 0.0386


  0%|▏                                                                                 | 2/680 [00:04<29:49,  2.64s/it]

...

 46%|████████████████████████████████████                                          | 314/680 [45:15<1:00:58, 10.00s/it]

1/1 [==============================] - 0s 3ms/step - loss: 0.1930 - mse: 0.1930


 46%|█████████████████████████████████████                                           | 315/680 [45:24<58:17,  9.58s/it]

1/1 [==============================] - 0s 3ms/step - loss: 0.1545 - mse: 0.1545


 46%|█████████████████████████████████████▏                                          | 316/680 [45:29<49:42,  8.19s/it]

In [191]:
model.save("capstone-Model-V1.h5")

In [185]:
np.array(ed.iloc[0][:-1])

array([-0.2563399970531464, 0.3745700120925903, -0.004352000076323748,
       -0.03136400133371353, 0.4970999956130981, 0.5001900196075441,
       -0.3169000148773193, -0.029404999688267708, -0.22394999861717224,
       2.38100004196167, -0.8131800293922424, 0.27068999409675604,
       0.3430800139904022, -0.07821600139141083, 0.26739999651908875,
       -0.5194100141525269, -0.31679001450538635, 0.8490999937057495,
       0.02203400060534477, -0.5310500264167786, 0.08698499947786331,
       -0.11784999817609787, 0.2147900015115738, 0.03904300183057785,
       -0.09996099770069122, 0.1139099970459938, -0.21740999817848208,
       -0.3270399868488312, -0.2949100136756897, -0.0860000029206276,
       -0.243819996714592, 0.09882400184869766, -0.022630000486969948,
       0.2913399934768677, 0.2726399898529053, 0.33285000920295715,
       0.045503001660108566, 0.09840700030326843, -0.185139998793602,
       -0.4682100117206574, 0.09139999747276306, -0.022836999967694286,
       -0.24865999

In [209]:
from numpy import dot
from numpy.linalg import norm
def cosine_sim(a):
    word=""
    res=[]
    cosine_score=0
    for i in range(ed_part.shape[0]):
        b=np.array(ed_part[i][:-1])
        result = dot(a, b)/(norm(a)*norm(b))
        res.append((result,ed_part[i][-1]))
    return res

In [210]:
x_train_pred=[]
x_train_score=[]
res=[]
for i in tqdm(range(5)):
    x_train_ind=np.expand_dims(x_train[i],axis=0)
    pred=model.predict(x_train_ind)
    pred=cosine_sim(pred)
    res.append(pred)
    
    
    

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.58s/it]


In [212]:
for i in res:
    i.sort(key = lambda x: x[0][0]) 

In [216]:
print(y_words_train[2])
res[2]

book


[(array([0.25844901148780214], dtype=object), 'son'),
 (array([0.26085421261582276], dtype=object), 'deaf'),
 (array([0.2843024915230523], dtype=object), 'secretary'),
 (array([0.2864904527491037], dtype=object), 'language'),
 (array([0.2902466859777261], dtype=object), 'cheat'),
 (array([0.2970657714601745], dtype=object), 'bowling'),
 (array([0.30036652603091446], dtype=object), 'corn'),
 (array([0.3046350543630751], dtype=object), 'cousin'),
 (array([0.3129902050502918], dtype=object), 'accident'),
 (array([0.3287390459343879], dtype=object), 'graduate'),
 (array([0.33636912065505853], dtype=object), 'basketball'),
 (array([0.3373191754646766], dtype=object), 'jacket'),
 (array([0.3375668129269054], dtype=object), 'cow'),
 (array([0.34627192569763166], dtype=object), 'computer'),
 (array([0.3478007208631443], dtype=object), 'family'),
 (array([0.35325350902813324], dtype=object), 'bed'),
 (array([0.3536270917002307], dtype=object), 'thin'),
 (array([0.35404478362533864], dtype=objec

In [204]:
x_train_ind=np.expand_dims(x_train[678],axis=0)
pred=model.predict(x_train_ind)
pred

array([[ 0.00460554,  0.4380381 , -0.24299075, -0.10257151,  0.01373655,
        -0.00272053,  0.01670466,  0.078439  ,  0.3024299 ,  0.9979546 ,
        -0.57699573, -0.20117456,  0.18793559, -0.24805453,  0.22190115,
         0.03420049, -0.23084016,  0.39901918,  0.01403801, -0.42293316,
         0.28125605, -0.02910172,  0.13610373, -0.2881159 ,  0.06241359,
         0.20312165, -0.359916  , -0.14180677,  0.06182242, -0.1978152 ,
        -0.00999522, -0.12556468,  0.43167183,  0.02578797, -0.29159445,
         0.35750997, -0.13816366,  0.27572468, -0.20251472,  0.05554481,
         0.09289262,  0.08085734, -0.23597078,  0.02994137, -0.12863539,
         0.27133057, -0.02173715, -0.0906937 ,  0.18245162, -0.22044195,
        -0.15967646,  0.11646146,  0.08221201, -0.17328668,  0.07844298,
         0.07002888, -0.31706235,  0.01129811,  0.30916712, -0.10759182,
        -0.6326554 , -0.15733069,  0.10806547, -0.14990124,  0.00380489,
        -0.2439765 , -0.3313536 ,  0.18459202,  0.0

In [160]:
x_test_pred=[]
x_test_score=[]
for i in tqdm(range(x_test.shape[0])):
    x_test_ind=np.expand_dims(x_test[i],axis=0)
    pred=model.predict(x_test_ind)
    word,score=cosine_sim(pred)
    x_test_pred.append(word)
    x_test_score.append(score)
    
    
    

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:36<00:00,  1.21s/it]


In [161]:
x_val_pred=[]
x_val_score=[]
for i in tqdm(range(x_val.shape[0])):
    x_val_ind=np.expand_dims(x_val[i],axis=0)
    pred=model.predict(x_val_ind)
    word,score=cosine_sim(pred)
    x_val_pred.append(word)
    x_val_score.append(score)
    
    
    

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:51<00:00,  1.39s/it]


In [195]:
x_val_pred=np.asarray(x_val_pred)
x_test_pred=np.asarray(x_test_pred)
x_train_pred=np.asarray(x_train_pred)

In [196]:
x_train_pred

array(['thursday', 'thursday', 'thursday', ..., 'thursday', 'thursday',
       'thursday'], dtype='<U8')

In [197]:
(x_train_pred==y_words_train).sum()/x_train_pred.shape[0]*100

0.9478672985781991

In [168]:
(x_test_pred==y_words_test).sum()/x_test_pred.shape[0]*100

13.333333333333334

In [169]:
(x_val_pred==y_words_val).sum()/x_val_pred.shape[0]*100

10.81081081081081

AttributeError: 'dict' object has no attribute 'sort'